In [1]:
!pip install category_encoders &> /dev/null

In [2]:
# Importing libraries
# Numpy for array operations and also to do calculations
import numpy as np
# Pandas for load dataset and manipulation of tabular data
import pandas as pd
# For plotting
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff
import matplotlib.pyplot as plt
%matplotlib inline

#from matplotlib import rcParams
#import matplotlib.gridspec as gridspec
#from IPython.core.pylabtools import figsize
#from datasist.structdata import detect_outliers

# For importing dataset from url
import urllib.request
# To interact with the underlying Operating System
import os
# For numerical computation and ML modeling phase
#import tensorflow as tf
# to manipulate date and time
#from datetime import datetime, timedelta
# Generate words cloud
#from wordcloud import WordCloud
# Import Images to notebook
#from IPython.display import Image
# For statistical analysis
from scipy import stats
# Set warnings to be ignored
import warnings
warnings.filterwarnings("ignore")

In [3]:

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler
from category_encoders import OrdinalEncoder
from category_encoders import BinaryEncoder
from sklearn.model_selection import cross_validate
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn import set_config
from sklearn.metrics import r2_score,mean_absolute_error

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Run this code to load data from google drive (if you are on google cloud colab)
df = pd.read_csv('/content/drive/MyDrive/Datasets/cleaned_autos.csv')

In [9]:
# Use this line of code to load data from local machine/folder
#df = df = pd.read_csv('cleaned_autos.csv')

In [10]:
df.head()

,dateCrawled,seller,offerType,price,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,lastSeen
0,2016-03-17 16:54:04,privat,Angebot,1500,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,2016-03-17 17:40:17
1,2016-03-31 17:25:20,privat,Angebot,3600,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,2016-04-06 10:17:21
2,2016-04-01 20:48:51,privat,Angebot,2200,cabrio,2004,manuell,109,2_reihe,150000,8,benzin,peugeot,nein,2016-04-01 00:00:00,2016-04-05 18:18:39
3,2016-03-26 19:54:18,privat,Angebot,2000,limousine,2004,manuell,105,3_reihe,150000,12,benzin,mazda,nein,2016-03-26 00:00:00,2016-04-06 10:45:34
4,2016-04-07 10:06:22,privat,Angebot,2799,kombi,2005,manuell,140,passat,150000,12,diesel,volkswagen,ja,2016-04-07 00:00:00,2016-04-07 10:25:17


In [11]:
df.shape

(197717, 16)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197717 entries, 0 to 197716
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   dateCrawled          197717 non-null  object
 1   seller               197717 non-null  object
 2   offerType            197717 non-null  object
 3   price                197717 non-null  int64 
 4   vehicleType          197717 non-null  object
 5   yearOfRegistration   197717 non-null  int64 
 6   gearbox              197717 non-null  object
 7   powerPS              197717 non-null  int64 
 8   model                197717 non-null  object
 9   kilometer            197717 non-null  int64 
 10  monthOfRegistration  197717 non-null  int64 
 11  fuelType             197717 non-null  object
 12  brand                197717 non-null  object
 13  notRepairedDamage    197717 non-null  object
 14  dateCreated          197717 non-null  object
 15  lastSeen             197717 non-nu

In [13]:
df.describe()

,price,yearOfRegistration,powerPS,kilometer,monthOfRegistration
count,197717.000000,197717.000000,197717.000000,197717.000000,197717.000000
mean,6035.413156,2004.058609,121.327529,123952.568570,6.281746
std,4668.025997,6.042001,49.274865,38744.157093,3.429715
min,1001.000000,1923.000000,0.000000,5000.000000,0.000000
25%,2390.000000,2001.000000,87.000000,100000.000000,3.000000
50%,4500.000000,2005.000000,116.000000,150000.000000,6.000000
75%,8499.000000,2008.000000,150.000000,150000.000000,9.000000
max,21210.000000,2018.000000,252.000000,150000.000000,12.000000


In [14]:
df.columns

Index(['dateCrawled', 'seller', 'offerType', 'price', 'vehicleType',
       'yearOfRegistration', 'gearbox', 'powerPS', 'model', 'kilometer',
       'monthOfRegistration', 'fuelType', 'brand', 'notRepairedDamage',
       'dateCreated', 'lastSeen'],
      dtype='object')

In [15]:
df = df.drop(['dateCrawled','yearOfRegistration','monthOfRegistration','dateCreated', 'lastSeen'], axis=1)

In [16]:
df.columns

Index(['seller', 'offerType', 'price', 'vehicleType', 'gearbox', 'powerPS',
       'model', 'kilometer', 'fuelType', 'brand', 'notRepairedDamage'],
      dtype='object')

In [17]:
df.powerPS.describe()

count    197717.000000
mean        121.327529
std          49.274865
min           0.000000
25%          87.000000
50%         116.000000
75%         150.000000
max         252.000000
Name: powerPS, dtype: float64

In [18]:
print(df.seller.unique())
print(df.offerType.unique())
print(df.gearbox.unique())
print(df.notRepairedDamage.unique())
print("")
print(df.vehicleType.unique())
print(df.fuelType.unique())
print("")
print(df.model.unique())
print("")
print(df.brand.unique())


['privat' 'gewerblich']
['Angebot' 'Gesuch']
['manuell' 'automatik']
['nein' 'ja']

['kleinwagen' 'cabrio' 'limousine' 'kombi' 'suv' 'bus' 'coupe' 'andere']
['benzin' 'diesel' 'lpg' 'andere' 'hybrid' 'cng' 'elektro']

['golf' 'fabia' '2_reihe' '3_reihe' 'passat' 'navara' 'twingo' 'c_max'
 'a_klasse' 'scirocco' '5er' 'civic' 'transporter' 'e_klasse' '3er'
 'andere' 'one' '1er' 'b_klasse' 'fortwo' 'a8' 'jetta' 'c_klasse' 'micra'
 'vito' 'sprinter' 'forester' 'fiesta' 'scenic' 'a1' 'combo' 'focus' 'tt'
 'astra' 'a6' 'jazz' 'polo' 'slk' '80' '147' 'z_reihe' 'ibiza' 'eos' 'a4'
 'touran' 'getz' 'insignia' 'megane' '7er' 'a3' 'lupo' 'clio' 'berlingo'
 'tiguan' 'mustang' 'sharan' '6_reihe' 'c4' 'panda' 'up' 'i_reihe' 'ceed'
 '5_reihe' 'yeti' 'octavia' 'zafira' 'mii' 'rx_reihe' 'corsa' '6er'
 'punto' 'fox' 'vectra' 'matiz' 'beetle' 'rio' 'touareg' 'logan' 'cuore'
 's_max' 'modus' 'a2' 'a5' 'galaxy' 'c3' 'viano' 'mondeo' 'avensis'
 'roomster' 'sl' 'kaefer' 'santa' 'cooper' 'sportage' 'caddy' 'cl

In [19]:
len(df.model.unique())

247

In [20]:
# Splitting the data
X = df.drop('price', axis=1)
y = df['price']

In [21]:
X.columns

Index(['seller', 'offerType', 'vehicleType', 'gearbox', 'powerPS', 'model',
       'kilometer', 'fuelType', 'brand', 'notRepairedDamage'],
      dtype='object')

In [22]:
from category_encoders import TargetEncoder
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

In [23]:
Encoder = ColumnTransformer(
    transformers=[
        ("BE", BinaryEncoder(), ['seller', 'offerType','gearbox','notRepairedDamage']),
         ('TE', TargetEncoder(),['vehicleType', 'fuelType','model', 'brand'])],
    remainder = "passthrough")

In [24]:
steps = []
steps.append(("Encoder" , Encoder))
steps.append(("Scaler" , StandardScaler()))
steps.append(("Model" , KNeighborsRegressor()))
pipeline = Pipeline(steps=steps)

In [25]:
kfold = KFold(n_splits=5, shuffle=True, random_state=0)
results = cross_validate(pipeline , X ,y , cv = kfold , scoring="r2" , return_train_score=True)


In [26]:
results["train_score"].mean()

0.7999958147342359

In [27]:
results["test_score"].mean()

0.7399488138879665

In [28]:
models = list()
#models.append(("LR" , LinearRegression()))
models.append(("KNN" , KNeighborsRegressor()))
models.append(("DT" , DecisionTreeRegressor()))
models.append(("RF" , RandomForestRegressor()))
models.append(("xg" , XGBRegressor()))

In [29]:
for model in models:
    steps = list()
    steps.append(("Encoder" , Encoder))
    steps.append(("Scaler" , RobustScaler()))
    steps.append(model)
    pipeline = Pipeline(steps = steps)
    kfold = KFold(n_splits=5, shuffle=True, random_state=0)
    scores = cross_validate(pipeline , X , y , scoring="r2"  ,cv = kfold , return_train_score=True)
    print(model[0])
    print("Train_r2" , scores["train_score"].mean() )
    print("-" * 10)
    print("Test_r2" , scores["test_score"].mean())
    print("-" * 20)
    print("\n")

LR
Train_r2 0.5691211563019787
----------
Test_r2 -3882084233246101.0
--------------------


DT
Train_r2 0.8873181381267141
----------
Test_r2 0.7427887351800833
--------------------


RF
Train_r2 0.8776779317503944
----------
Test_r2 0.7901502902467288
--------------------


xg
Train_r2 0.787714244631107
----------
Test_r2 0.7688327125907259
--------------------




best model is XGregressor

In [30]:
steps = list()
steps.append(("Encoder" , Encoder))
steps.append(("Scaler" , RobustScaler()))
steps.append(("xg" , XGBRegressor()))
pipeline = Pipeline(steps = steps)
kfold = KFold(n_splits=5, shuffle=True, random_state=0)
scores = cross_validate(pipeline , X , y , scoring="r2"  ,cv = 5 , return_train_score=True , return_estimator=True)

In [31]:
scores["estimator"][0]["xg"].get_params()

{'objective': 'reg:squarederror',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'device': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': None,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': None,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [32]:
param_grid = {
    "xg__learning_rate": [0.2],
    "xg__max_depth": [10],
    "xg__n_estimators": [240],
    "xg__reg_lambda" : [4]

}
# best parameters are 0.2 10 240 4 with train & test 0.8666 0.8020

In [33]:
steps = []
steps.append(("Encoder" , Encoder))
steps.append(("Scaler" , RobustScaler()))
steps.append(("xg" , XGBRegressor()))
pipeline = Pipeline(steps=steps)
kfold = KFold(n_splits=5, shuffle=True, random_state=0)
grid_search = GridSearchCV(estimator=pipeline , param_grid=param_grid , cv =kfold ,scoring="r2" , return_train_score=True , n_jobs = -1)
grid_search.fit(X,y)


GridSearchCV(cv=KFold(n_splits=5, random_state=0, shuffle=True),
             estimator=Pipeline(steps=[('Encoder',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('BE',
                                                                         BinaryEncoder(),
                                                                         ['seller',
                                                                          'offerType',
                                                                          'gearbox',
                                                                          'notRepairedDamage']),
                                                                        ('TE',
                                                                         TargetEncoder(),
                                                                         ['vehicleType',
                                                                          'fuelType',
                                                                          'model',
                                                                          'brand'])])),
                                       ('Scaler', RobustScaler()),
                                       ('xg',
                                        XGBRegressor(base_...
                                                     max_delta_step=None,
                                                     max_depth=None,
                                                     max_leaves=None,
                                                     min_child_weight=None,
                                                     missing=nan,
                                                     monotone_constraints=None,
                                                     multi_strategy=None,
                                                     n_estimators=None,
                                                     n_jobs=None,
                                                     num_parallel_tree=None,
                                                     random_state=None, ...))]),
             n_jobs=-1,
             param_grid={'xg__learning_rate': [0.2], 'xg__max_depth': [10],
                         'xg__n_estimators': [240], 'xg__reg_lambda': [3]},
             return_train_score=True, scoring='r2')

In [34]:
grid_search.best_params_

{'xg__learning_rate': 0.2,
 'xg__max_depth': 10,
 'xg__n_estimators': 240,
 'xg__reg_lambda': 3}

In [35]:
grid_search.cv_results_["mean_train_score"].mean()

0.8643942701116154

In [36]:
grid_search.cv_results_["mean_test_score"].mean()

0.8015394206372184